In [25]:
import re
from typing import List, Dict, NamedTuple, Optional, Set

BagsDict  =Dict[str,Dict[str,int]]

class Bag(NamedTuple):
    bag_type: str
    num: int = 1
           
    @staticmethod
    def parse_text(text:str):
        """
        3 pale aqua bag or duffy douche bag
        """
        REGEX = "^ ?(?P<num>[0-9])? (?P<type>[a-z]+ [a-z]+) bags?"
        m = re.match(REGEX, text)
        return Bag(num = int(m.group('num')), 
                   bag_type = m.group('type'))          

class Node(NamedTuple):
    parent: Bag
    child: Dict[str, int] = None
        
    @staticmethod
    def parse_row(row:str)-> Bag:
        """
        handle input row day 6
        """
        contain = row.split("contain")
        contain0 = "1 " + contain[0] # not so clean...
        if 'no other bags' in contain[1]:
            return Node(parent = Bag.parse_text(contain0))
        else:
            child = {}
            commas = contain[1].split(",")
            for text in commas:
                b = Bag.parse_text(text)
                child[b.bag_type] = b.num
        main = Bag.parse_text(contain0)
        return Node(parent= main, child=child)

def get_bags(raw:str)-> List[Node]:
    return [
        Node.parse_row(r)
        for r in raw.split("\n")
    ]

def get_dict(Nodes: List[Node])-> BagsDict:
    return {
        node.parent.bag_type:node.child
        for node in Nodes
    }

def get_immediate(n_dict: BagsDict, target:str)-> Set[str]:
    immediate = set()
    for bag, child in ds.items():
        if child and target in child.keys():
            immediate.add(bag)
    return immediate
            

def num_bags_contain(n_dict:BagsDict, target:str) -> int:
    bags = set()
    target_set = {target}
    while True:
        target = target_set.pop() 
        immediate_bags = get_immediate(n_dict=ds, target=target)
        for bag in immediate_bags:
            bags.add(bag)
            target_set.add(bag)
        if not target_set:
            break
    return len(bags)

TEST = """light red bags contain 1 bright white bag, 2 muted yellow bags.
dark orange bags contain 3 bright white bags, 4 muted yellow bags.
bright white bags contain 1 shiny gold bag.
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
dark olive bags contain 3 faded blue bags, 4 dotted black bags.
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
faded blue bags contain no other bags.
dotted black bags contain no other bags."""

In [26]:
test = get_bags(raw=TEST)
ds = get_dict(Nodes=test)
target = 'shiny gold'
assert num_bags_contain(n_dict=ds, target=target) == 4

In [27]:
with open('puzzle_inputs/day07_01.txt') as f:
    target = 'shiny gold'
    RAW = f.read()
    nodes = get_bags(raw=RAW)
    ds = get_dict(Nodes=nodes)
    print(num_bags_contain(n_dict=ds, target=target))

128


Part 2

   Consider again your shiny gold bag and the rules from the above example:

    faded blue bags contain 0 other bags.
    dotted black bags contain 0 other bags.
    vibrant plum bags contain 11 other bags: 5 faded blue bags and 6 dotted black bags.
    dark olive bags contain 7 other bags: 3 faded blue bags and 4 dotted black bags.

So, a single shiny gold bag must contain 1 dark olive bag (and the 7 bags within it) plus 2 vibrant plum bags (and the 11 bags within each of those): 1 + 1 x 7 + 2 + 2 x 11 = 32 bags!

In [28]:
TEST = """shiny gold bags contain 2 dark red bags.
dark red bags contain 2 dark orange bags.
dark orange bags contain 2 dark yellow bags.
dark yellow bags contain 2 dark green bags.
dark green bags contain 2 dark blue bags.
dark blue bags contain 2 dark violet bags.
dark violet bags contain no other bags."""

In [29]:
def total_bags(n_dict:BagsDict, target:str) -> int:
    total_bags = 0
    stack = [(target, 1)]
    while stack:
        next_bag, count = stack.pop()
        children = ds[next_bag]
        if not children:
            continue
        for bag, num in children.items():
            total_bags += count * num
            stack.append((bag, num * count))
    return total_bags
        

In [30]:
bags = get_bags(TEST)
ds = get_dict(bags)
target = 'shiny gold'
assert total_bags(n_dict=ds, target=target) == 126


In [31]:
with open('puzzle_inputs/day07_01.txt') as f:
    target = 'shiny gold'
    RAW = f.read()
    nodes = get_bags(raw=RAW)
    ds = get_dict(Nodes=nodes)
    print(total_bags(n_dict=ds, target=target))

20189
